<a href="https://colab.research.google.com/github/Prasad14-hub/LLM-Project/blob/main/Prasad_Gavhane_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installing all the required libraries

!pip install langchain
!pip install torch
!pip install transformers
!pip install sentence-transformers
!pip install pypdf
!pip install faiss-gpu
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.6 MB/s eta 0:00:00


In [ ]:
import transformers
import torch

In [ ]:
#Importing all the required Classes

from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings


In [ ]:
#Loading the pdf file

loader=PyPDFLoader("Prasad Gavhane_Resume (3).pdf")
documents=loader.load()


In [ ]:
#Splitting the text into chucks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [ ]:
#Initializing embeddings

from langchain.embeddings import HuggingFaceEmbeddings

modelPath = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device':'cuda'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#Creating Vector Database
vector_db = FAISS.from_documents(docs, embeddings)
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

In [ ]:
#model = "anakin87/zephyr-7b-alpha-sharded"
model="stabilityai/stablelm-zephyr-3b"

tokenizer = AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipeline = transformers.pipeline(
    "text-generation",        # Specifying the task
    model=model,              # Providing the pre-trained model
    tokenizer=tokenizer,      # Setting tockenizer for text procressing
    torch_dtype=torch.bfloat16,  #Specifying the torch type
    trust_remote_code=True,   # Enabling trust in remote code
    device_map="auto",        # Using automatic device mapping
    max_length=1000,          # Setting the maximum length of generated text tokens
    top_k=10,                 # Specifying the top-k sampling parameter with the highest probabilites
    num_return_sequences=1,   # Number of generated sequences to return
    eos_token_id=tokenizer.eos_token_id, # Specifying the end-of-sequence token ID
    do_sample=True, #Should be set True if we use top_k or top_p sampling
)

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.59G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
#Creating a custom HuggingFacePipeline instance with additional model configuration
llm = HuggingFacePipeline(
    pipeline=pipeline,         # Providing the pre-configured HuggingFace pipeline
    model_kwargs={'temperature': 0.1}  # Setting additional model configuration
)

In [ ]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

In [ ]:
query="What are different certifications does Prasad Gavhane have?"
out = chain.invoke(query)
print(out['result'])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 Prasad Gavhane has passed the Google Data Analytics exam by Google, Python 101 for Data Science, Data Analysis with Python by IBM, and Problem Solving using C++. He also has SQL (Intermediate) certification from Hacker Rank. Additionally, he has taken the Microsoft Azure -900 examination and achieved a score of 895/1000.


In [ ]:
#Trying out different prompts
from langchain import PromptTemplate, LLMChain

template="""Question:{question},
            Answer:Let's give detailed answers."""

prompt=PromptTemplate(template=template,input_variables=["question"])

In [ ]:
chain=LLMChain(prompt=prompt,llm=llm)
out=chain.invoke("Give me a recipe for Masala Dosa.")
print(out["text"])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 A step by step guide on how to prepare Masala Dosa at home.

Ingredients:
For the Dosa:
- 2 cups of rice flour
- 1/2 cup of urad dal
- Water as required
- Salt to taste
- Oil or ghee for cooking

For the Masala:
- 1 onion, finely chopped
- 1 tomato, finely chopped
- 1 Green chili, finely chopped
- 1 inch piece of ginger, grated
- 1/2 tsp cumin seeds
- 1 tsp mustard seeds
- 1 tsp turmeric powder
- 1 tsp red chili powder
- Salt to taste
- 1 tbsp coriander leaves, finely chopped
- 2 tbsp oil
- Water as required

Instructions:

1. Soak the rice flour and urad dal in water for at least 2 hours.
2. Drain the water and grind the soaked ingredients to a fine paste using water to make a smooth batter, adding salt to it. Keep aside.
3. Heat a non-stick pan or tawa on medium flame. Pour a ladle full of the batter in the center and spread it in a circular motion, using the back of the ladle or a dosa cutlet maker, to make a thin circular dosa. Add a little oil or ghee around the edges to cook it 

In [ ]:
chat = [
       {
        "role": "system",
        "content": "You are a friendly chatbot who always give short and precise answers"
       },
       {
        "role": "user",
        "content": "What is the capital of Maharashtra?"
       }
       ]

prompt = pipeline.tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
#add_generation_prompt=True is to indicate the start of a bot response
print(prompt)

<|system|>
You are a friendly chatbot who always give short and precise answers<|endoftext|>
<|user|>
What is the capital of Maharashtra?<|endoftext|>
<|assistant|>



In [ ]:
outputs = pipeline(prompt)
print(outputs[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|system|>
You are a friendly chatbot who always give short and precise answers<|endoftext|>
<|user|>
What is the capital of Maharashtra?<|endoftext|>
<|assistant|>
The capital of Maharashtra is Mumbai.

Step 1: Recognize the user's question is asking for the capital of Maharashtra.
Step 2: Recall that Mumbai is the capital city of the Indian state of Maharashtra.
Step 3: Present the answer in a concise manner.
